# Illustration for custom layer: Softmax_cosine_sim


In [2]:
import random
import numpy as np
from random import shuffle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.activations import softmax 

class SoftmaxCosineSim(keras.layers.Layer):
    # ==============================================================================
    # Code modified from NT-XENT-loss: 
    # https://github.com/google-research/simclr/blob/master/objective.py
    # ==============================================================================
    # coding=utf-8
    # Copyright 2020 The SimCLR Authors.
    #
    # Licensed under the Apache License, Version 2.0 (the "License");
    # you may not use this file except in compliance with the License.
    # You may obtain a copy of the License at
    #
    #     http://www.apache.org/licenses/LICENSE-2.0
    #
    # Unless required by applicable law or agreed to in writing, software
    # distributed under the License is distributed on an "AS IS" BASIS,
    # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    # See the License for the specific simclr governing permissions and
    # limitations under the License.
    # ==============================================================================
    def __init__(self, batch_size, feat_dim, temperature = 0.1, LARGE_NUM = 1e9):
        self.batch_size = batch_size
        self.feat_dim = feat_dim
        self.units = (batch_size, 4 * feat_dim)
        self.input_dim = [(None, feat_dim)] * (batch_size * 2)
        self.temperature = temperature
        self.LARGE_NUM = LARGE_NUM
        super(SoftmaxCosineSim, self).__init__()
        
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'batch_size' : self.batch_size,
            'feat_dim' : self.feat_dim,
            'units' : self.units,
            'input_dim' : self.input_dim,
            'temperature' : self.temperature,
            'LARGE_NUM' : self.LARGE_NUM,
        })
        return config    

    def call(self, inputs):
        # Function to perform tranformatiom: input -> output
        batch_size = len(inputs) // 2
        z1 = []
        z2 = []
        
        for index in range(batch_size):
            # 0 assumes that batch in generator is actually just 1
            z1.append(tf.math.l2_normalize(inputs[index][0], -1))
            z2.append(tf.math.l2_normalize(inputs[batch_size + index][0], -1))
        
        # Gather hidden1/hidden2 across replicas and create local labels.
        z1_large = z1
        z2_large = z2
                      
        # TODO: move to GENERATOR 
        labels = tf.one_hot(tf.range(batch_size), batch_size * 2)
        masks = tf.one_hot(tf.range(batch_size), batch_size)

        # Products of vectors of same side of network (z_i), count as negative examples
        logits_aa = tf.matmul(z1, z1_large, transpose_b=True) / self.temperature
        print(f"logits_aa \n {logits_aa} \n")

        # Values on the diagonal are put equal to a very small value -> exclude product between 2 identical values
        logits_aa = logits_aa - masks * self.LARGE_NUM
        print(f"logits_aa after mask:: \n {logits_aa} \n")

        # Similar as aa
        logits_bb = tf.matmul(z2, z2_large, transpose_b=True) / self.temperature
        logits_bb = logits_bb - masks * self.LARGE_NUM


        # Comparison between two sides of the network (z_i and z_j) -> diagonal should be as close as possible to 1
        logits_ab = tf.matmul(z1, z2_large, transpose_b=True) / self.temperature
        print(f"logits_ab: \n {logits_ab} \n")
        # Similar as ba
        logits_ba = tf.matmul(z2, z1_large, transpose_b=True) / self.temperature
        
        print(tf.concat([logits_ab, logits_aa], 1))
        
        part1 = softmax(tf.concat([logits_ab, logits_aa], 1))
        part2 = softmax(tf.concat([logits_ba, logits_bb], 1))
        output = tf.concat([part1, part2], 1)
        
        return output 

def tf_round_decimal(x, decimals = 0):
    multiplier = tf.constant(10**decimals, dtype=x.dtype)
    return tf.round(x * multiplier) / multiplier

## 1. Strictly ordered feature vectors

In [3]:
batch_size = 3
feat_dim = 5

In [4]:
hidden_ordered = [tf.convert_to_tensor(np.asarray([[1,0,0,0,0]]).astype('float32')), 
          tf.convert_to_tensor(np.asarray([[0,0,0,0,1]]).astype('float32')),
          tf.convert_to_tensor(np.asarray([[0,0,1,0,0]]).astype('float32')),
          tf.convert_to_tensor(np.asarray([[1,0,0,0,0]]).astype('float32')), 
          tf.convert_to_tensor(np.asarray([[0,0,0,0,1]]).astype('float32')), 
          tf.convert_to_tensor(np.asarray([[0,0,1,0,0]]).astype('float32'))]

In [5]:
SoftmaxCosineSim_layer = SoftmaxCosineSim(batch_size = batch_size, feat_dim = feat_dim)
y = SoftmaxCosineSim_layer(hidden_ordered)
print(f" \n output: \n \n {tf_round_decimal(y,4)}")

logits_aa 
 [[10.  0.  0.]
 [ 0. 10.  0.]
 [ 0.  0. 10.]] 

logits_aa after mask:: 
 [[-1.e+09  0.e+00  0.e+00]
 [ 0.e+00 -1.e+09  0.e+00]
 [ 0.e+00  0.e+00 -1.e+09]] 

logits_ab: 
 [[10.  0.  0.]
 [ 0. 10.  0.]
 [ 0.  0. 10.]] 

tf.Tensor(
[[ 1.e+01  0.e+00  0.e+00 -1.e+09  0.e+00  0.e+00]
 [ 0.e+00  1.e+01  0.e+00  0.e+00 -1.e+09  0.e+00]
 [ 0.e+00  0.e+00  1.e+01  0.e+00  0.e+00 -1.e+09]], shape=(3, 6), dtype=float32)
 
 output: 
 
 [[0.9998 0.     0.     0.     0.     0.     0.9998 0.     0.     0.
  0.     0.    ]
 [0.     0.9998 0.     0.     0.     0.     0.     0.9998 0.     0.
  0.     0.    ]
 [0.     0.     0.9998 0.     0.     0.     0.     0.     0.9998 0.
  0.     0.    ]]


## 2. Shuffled feature vectors

In [6]:
hidden_shuffled = [tf.convert_to_tensor(np.asarray([[1,0,0,0,0]]).astype('float32')), 
                   tf.convert_to_tensor(np.asarray([[0,0,0,0,1]]).astype('float32')),
                   tf.convert_to_tensor(np.asarray([[0,0,1,0,0]]).astype('float32')),
                   tf.convert_to_tensor(np.asarray([[0,0,1,0,0]]).astype('float32')),
                   tf.convert_to_tensor(np.asarray([[1,0,0,0,0]]).astype('float32')),
                   tf.convert_to_tensor(np.asarray([[0,0,0,0,1]]).astype('float32'))]
for h in hidden_shuffled:
    print(h.numpy()[0])

[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[0. 0. 1. 0. 0.]
[0. 0. 1. 0. 0.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]


#### => This should result on 1 on index (1,2), (2,3), (3,1)

In [7]:
y = SoftmaxCosineSim_layer(hidden_shuffled)
print(f" \n output: \n \n {tf_round_decimal(y,4)}")

logits_aa 
 [[10.  0.  0.]
 [ 0. 10.  0.]
 [ 0.  0. 10.]] 

logits_aa after mask:: 
 [[-1.e+09  0.e+00  0.e+00]
 [ 0.e+00 -1.e+09  0.e+00]
 [ 0.e+00  0.e+00 -1.e+09]] 

logits_ab: 
 [[ 0. 10.  0.]
 [ 0.  0. 10.]
 [10.  0.  0.]] 

tf.Tensor(
[[ 0.e+00  1.e+01  0.e+00 -1.e+09  0.e+00  0.e+00]
 [ 0.e+00  0.e+00  1.e+01  0.e+00 -1.e+09  0.e+00]
 [ 1.e+01  0.e+00  0.e+00  0.e+00  0.e+00 -1.e+09]], shape=(3, 6), dtype=float32)
 
 output: 
 
 [[0.     0.9998 0.     0.     0.     0.     0.     0.     0.9998 0.
  0.     0.    ]
 [0.     0.     0.9998 0.     0.     0.     0.9998 0.     0.     0.
  0.     0.    ]
 [0.9998 0.     0.     0.     0.     0.     0.     0.9998 0.     0.
  0.     0.    ]]


#### => Indeed, 1 on index (1,2), (2,3), (3,1)

# 3. Illustration of temperature importance

In [9]:
SoftmaxCosineSim_layer = SoftmaxCosineSim(batch_size = batch_size, feat_dim = feat_dim, temperature = 1)
y = SoftmaxCosineSim_layer(hidden_ordered)
print(f" \n output: \n \n {tf_round_decimal(y,3)}")

logits_aa 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] 

logits_aa after mask:: 
 [[-1.e+09  0.e+00  0.e+00]
 [ 0.e+00 -1.e+09  0.e+00]
 [ 0.e+00  0.e+00 -1.e+09]] 

logits_ab: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] 

tf.Tensor(
[[ 1.e+00  0.e+00  0.e+00 -1.e+09  0.e+00  0.e+00]
 [ 0.e+00  1.e+00  0.e+00  0.e+00 -1.e+09  0.e+00]
 [ 0.e+00  0.e+00  1.e+00  0.e+00  0.e+00 -1.e+09]], shape=(3, 6), dtype=float32)
 
 output: 
 
 [[0.405 0.149 0.149 0.    0.149 0.149 0.405 0.149 0.149 0.    0.149 0.149]
 [0.149 0.405 0.149 0.149 0.    0.149 0.149 0.405 0.149 0.149 0.    0.149]
 [0.149 0.149 0.405 0.149 0.149 0.    0.149 0.149 0.405 0.149 0.149 0.   ]]
